In [ ]:
# The following was obtained from stackoverflow

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.common.exceptions import TimeoutException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import NoAlertPresentException
# import sys

# import unittest, time, re

# class Sel(unittest.TestCase):
#     def setUp(self):
#         self.driver = webdriver.Firefox()
#         self.driver.implicitly_wait(30)
#         self.base_url = "https://twitter.com"
#         self.verificationErrors = []
#         self.accept_next_alert = True
        
#     def test_sel(self):
#         driver = self.driver
#         delay = 3
#         driver.get(self.base_url + "/search?q=stckoverflow&src=typd")
#         driver.find_element_by_link_text("All").click()
#         for i in range(1,100):
#             self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(4)
#         html_source = driver.page_source
#         data = html_source.encode('utf-8')


# if __name__ == "__main__":
#     unittest.main()

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time, re

keyword = 'data analyst'
title = re.sub(' ', '%20', keyword.lower()) # This is used for building url
keyword = re.sub(' ', '-', keyword.lower())

# Not using context manager, because we may need to close and restart if running into an authentication wall
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(f"https://www.linkedin.com/jobs/search/?distance=25&geoId=102454443&keywords={title}&location=Singapore&start=0")
html_source = driver.page_source
soup = BeautifulSoup(html_source,'html.parser')
retry_count = 0
# Sometimes we will run into an authentication wall, retry again until successful
while retry_count < 5 and "Sign Up | LinkedIn" in soup.find('title').text:
    print(f"\tRan into AuthWall, trying again in 5 secs...")
    driver.quit()
    time.sleep(5)
    driver = webdriver.Chrome()
    driver.get(f"https://www.linkedin.com/jobs/search/?distance=25&geoId=102454443&keywords={title}&location=Singapore&start=0")
    html_source = driver.page_source
    soup = BeautifulSoup(html_source,'html.parser')
    retry_count +=1

for i in range(1,100):
    time.sleep(2)
    webdriver.ActionChains(driver).scroll_by_amount(0, -10).perform() # Need to scroll up a little to trigger infinite scroll
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try: # Occasionally there will be a click to see more button appear, need to click on it, sometimes two clicks are needed
        element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located(
                (By.XPATH,'//*[@id="main-content"]/section[2]/button')))
        if element:
            driver.execute_script("arguments[0].click()", element)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click()", element)
    except Exception as e:
        print(e)
    
html_source = driver.page_source
driver.quit()

	Ran into AuthWall, trying again in 5 secs...


In [23]:
def custom_selector(tag):
        '''
        Helper function used to identify a href tag with belongs to the job link
        Inputs:
            soup tag
        Returns soup tag
        '''
        return tag.name == "a" and tag.has_attr("href") and keyword in tag.get('href')

data = {}
soup = BeautifulSoup(html_source,'html.parser')
tags = soup.find_all(custom_selector)

# Get the link and jobid for each listed job
for tag in tags:
    link = tag.get('href')
    link = link.split('?')[0] # Tidy up the link to remove the trackingid
    if 'login' not in link:
        data.setdefault(link, 0)
        data[link] += 1
data

{'https://sg.linkedin.com/jobs/view/data-analyst-at-capgemini-3764392364': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-at-starbucks-coffee-singapore-3775365168': 1,
 'https://sg.linkedin.com/jobs/view/junior-data-analyst-business-intelligence-regional-operations-at-shopee-3727428385': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-planning-budgeting-sg-warehouse-at-shopee-3783416267': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-at-sea-3616637587': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-marketing-analytics-regional-brand-growth-marketing-at-shopee-3775405705': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-marketplace-product-at-shopee-3721375973': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-at-ninja-van-3772981858': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-chatbot-customer-service-2024-fresh-graduates-at-shopee-3792429156': 1,
 'https://sg.linkedin.com/jobs/view/data-analyst-marketplace-product-2024-fresh-graduates-at-shopee-377